# Hoja de trabajo 7
Jose Daniel Gomez Cabrera 21429

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
import time

# Carga de datos

In [ ]:
train = pd.read_csv("./train.csv")
